# MOVIE REVIEW SENTIMENT ANALYSIS

This dataset has been obtained from http://ai.stanford.edu/~amaas/data/sentiment/
this is a very large dataset with 12500 positive and 12500 negative results for training
and for test there are separate set of records to process with 

each review is provided as single file. we have to load them into a pandas dataframe. 
Let's create a function to load the data into data frame. Data doesn't have labels in them but they are
seggregated in folders. We need to attach them the tags while loading a dataframe

##### We can use any kind of classifier, for this example svm linear classifier has been used. we can experiment with any kind of classifier and use the best 

In [101]:
import pandas as pd
import os as os
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

below function will help to load individual test files into dataframe and returns a dataframe with sentiment

In [55]:
def load_file(sent,path):
    full_path = path
    df=[]
    for filename in os.listdir(full_path):
        file=full_path+'/'+filename
        with open(file, encoding="utf8") as f:
            review=f.read()
            df.append(review)
    data = pd.DataFrame(df,columns=['review'])
    data['tag']=sent
    return data     

below code will load all individual files into a single data frame using the function created above

In [82]:
total=load_file('pos','C:/Users/sagar/Documents/movies/pos')
total=total.append(load_file('neg','C:/Users/sagar/Documents/movies/neg'))

In [63]:
total.shape

(25000, 2)

In [ ]:
#check the records for nulls and any spaces

In [64]:
total.isnull().sum()

review    0
tag       0
dtype: int64

In [66]:
spaces=[]
for row in total.itertuples():
    if row.review.isspace():
        spaces.append(rec.index)

shuffle the records as they are ordered while being loaded

In [89]:
total=total.sample(frac=1)
total.reset_index(drop=True,inplace=True)

In [90]:
total.head()

,review,tag
0,This is by far one of my favorite of the Ameri...,pos
1,because you can put it on fast forward and wat...,neg
2,I cannot begin to describe how amazing this mo...,pos
3,This movie is maybe the most touching and upli...,pos
4,"Walt Disney's ""The Rookie"" is based on the sto...",pos


#### obtain test and train datasets

In [94]:
X_train,X_test,y_train,y_test=train_test_split(total['review'],total['tag'],test_size=0.3,random_state=42)

create a pipeline for future purposes. we can include custom function as well in pipeline for handling
the input data. For example handling the stop words or lemmatizing etc.

In [98]:
text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])

In [99]:
text_clf.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [100]:
y_pred=text_clf.predict(X_test)

#### create a confusion matrix to understand the results

In [107]:
cm=pd.DataFrame(confusion_matrix(y_test,y_pred),columns=['pos','neg'],index=['pos','neg'])
cm

,pos,neg
pos,3348,404
neg,391,3357


In [109]:
print(classification_report(y_test,y_pred))
print('accuracy of the model is:', accuracy_score(y_test,y_pred))

             precision    recall  f1-score   support

        neg       0.90      0.89      0.89      3752
        pos       0.89      0.90      0.89      3748

avg / total       0.89      0.89      0.89      7500

accuracy of the model is: 0.894


check a sample positive feedback once to see the result

In [112]:
text_clf.predict(['Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". '])

array(['pos'], dtype=object)

check a sample negative feedback once to see the result

In [114]:
text_clf.predict(['Weak plot, predictable violence, only semi interesting characters. Like the writer (also one of the stars?) was fictionalizing'])

array(['neg'], dtype=object)

### Hurray!!! the model is working awesome!